In [10]:
import gensim
import pandas as pd

In [4]:
data = pd.read_json('http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz', lines=True)
data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [9]:
data.shape

(194439, 9)

### Simple Preprocessing & Tokenization
- The first thing to do for any data science task is to clean the data. For NLP, we apply various processing like converting all the words to lower case, trimming spaces, removing punctuations. This is something we will do over here too.

- Additionally, we can also remove stop words like 'and', 'or', 'is', 'the', 'a', 'an' and convert words to their root forms like 'running' to 'run'.

In [12]:
review_text = data['reviewText'].apply(gensim.utils.simple_preprocess)
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

### Training the Word2Vec Model
- Train the model for reviews. Use a window of size 10 i.e. 10 words before the present word and 10 words ahead. A sentence with at least 2 words should only be considered, configure this using min_count parameter.

- Workers define how many CPU threads to be used.

In [13]:
# initialize the model
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

In [14]:
# build vocabulary
model.build_vocab(review_text, progress_per=1000)

### Train the Word2Vec Model

In [15]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(61510112, 83868975)

### Save the Model
- Save the model so that it can be reused in other applications

In [16]:
model.save('./word2vec-amazon-cell-accessories-reviews-short.model')

### Finding Similar Words and Similarity between words

In [17]:
model.wv.most_similar('bad')

[('terrible', 0.6717463135719299),
 ('shabby', 0.6714062690734863),
 ('good', 0.6020506620407104),
 ('horrible', 0.5884097218513489),
 ('okay', 0.5467613339424133),
 ('funny', 0.5387152433395386),
 ('awful', 0.5385643243789673),
 ('legit', 0.5252166390419006),
 ('ok', 0.5245727300643921),
 ('fault', 0.5114441514015198)]

In [18]:
model.wv.similarity(w1='cheap', w2='inexpensive')

0.52360386

In [19]:
model.wv.similarity(w1='great', w2='great')

0.99999994